In [1]:
pip install --upgrade opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 825.8 kB/s eta 0:00:47
   ---------------------------------------- 0.1/38.8 MB 581.0 kB/s eta 0:01:07
   ---------------------------------------- 0.2/38.8 MB 985.7 kB/s eta 0:00:40
   ---------------------------------------- 0.3/38.8 MB 1.2 MB/s eta 0:00:32
   ---------------------------------------- 0.4/38.8 MB 1.4 MB/s eta 0:00:27
    --------------------------------------- 0.5/38.8 MB 1.7 MB/s eta 0:00:23
    --------------------------------------- 0.6/38.8 MB 1.8 MB/s eta 0:00:22
    --------------------------------------- 0.6/38.8 MB 1.7 MB/s eta 0:00:23
    --------------------------------------- 0.9/38.8 MB 1.9 MB/s eta 0:00:20
   - -------------------------------------- 1.2/38.8 MB 2.3 MB/s eta 0:00:17
   - -------------------------------------- 1.5/38.8 MB 2.7 MB/s eta 0:00:14
   - --

In [2]:
pip install --upgrade cvlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

In [4]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [5]:
def f_keepLargeComponents(I,th):
    R = np.zeros(I.shape)<0
    unique_labels = np.unique(I.flatten())
    for label in unique_labels:
        if label == 0:
            pass
        else:
            I2 = I==label
            if np.sum(I2)>th:
                R = R | I2
    return np.float32(255*R)

In [6]:
import cvlib as cv 
from cvlib.object_detection import draw_bbox
def f_saveSeq(C,counter,th,outPutPath):
    if len(C)<th:
        pass
    else:
        k=1
        for frame in C:
            imName = str(counter)+'_'+ str(k)+'.jpg'
            finalPath = os.path.join(outPutPath,imName)
            # bbox,labels,conf = cv.detect_common_objects(frame)
            # frame = draw_bbox(frame,bbox,labels,conf)
            cv2.imwrite(finalPath,frame)
            k += 1

In [12]:
# def f_saveSeq(C, counter, th, outPutPath):
#     if len(C) < th:
#         print("Warning: Not enough frames to save sequence.")
#         return

#     k = 1
#     for frame in C:
#         imName = f"{counter}_{k}.jpg"
#         finalPath = os.path.join(outPutPath, imName)
        
#         # Perform object detection
#         bbox, labels, conf = cv.detect_common_objects(frame, confidence=0.5)
#         # Check if objects were detected
#         if bbox:
#             # Draw bounding boxes on the frame
#             frame = draw_bbox(frame, bbox, labels, conf)
#             # Save the modified frame to disk
#             cv2.imwrite(finalPath, frame)
#             print(f"Saved {imName}")
#         else:
#             print(f"No objects detected in frame {k}, skipping.")
        
#         k += 1


In [13]:
def f_displaySeq(outPutPath):
    for imName in os.listdir(outPutPath):
        frame = cv2.imread(os.path.join(outPutPath,imName))
        frame = cv2.resize(frame,dsize=(600,400))
        cv2.imshow('Display',frame)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    cv2.destroyAllWindows()
    

In [9]:
pth = r'C:\Users\pc\OneDrive\Desktop\DIP project\Saved_new'
f_displaySeq(pth)

In [10]:
# imPath = r'C:\Users\pc\OneDrive\Desktop\DIP project\new_video'
imPath = r'C:\Users\pc\OneDrive\Desktop\DIP project\basketball'
fgModel = cv2.createBackgroundSubtractorMOG2()
leastNumOfFrames = 5
idx = []
C = []
counter = 0
outPutPath = r'C:\Users\pc\OneDrive\Desktop\DIP project\Saved_new'
for imName in os.listdir(imPath):
    counter += 1
    frame = cv2.imread(os.path.join(imPath,imName))
    frame = cv2.resize(frame,dsize=(600,400))
    fgmask = fgModel.apply(frame)
    k_r = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    fgmask = cv2.morphologyEx(np.float32(fgmask),cv2.MORPH_OPEN,k_r)
    num_labels,labels_im = cv2.connectedComponents(np.array(fgmask>0,np.uint8))
    fgmask = f_keepLargeComponents(labels_im,1000)
    if np.sum(fgmask)>0:
        idx.append(counter)
        C.append(frame)
    if len(idx) >= 2 and idx[-1] > idx[-2]+1:
        f_saveSeq(C,counter,leastNumOfFrames,outPutPath)
        idx = []
        C = []
    F = np.zeros(frame.shape,np.uint8)
    F[:,:,0],F[:,:,1],F[:,:,2] = fgmask, fgmask, fgmask
    F2 = np.hstack((frame, F))
    cv2.imshow('Display',F2)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
f_saveSeq(C,counter,leastNumOfFrames,outPutPath)
cv2.destroyAllWindows()